<a href="https://colab.research.google.com/github/dattali18/IR_Assignments/blob/main/Assignment.04/IR_04_extract_sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [3]:
!pip install pandas numpy torch transformers datasets scikit-learn tqdm

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
# get the data from the github repository
aj_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/A_J_word.csv?raw=true"
bbc_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/BBC_word.csv?raw=true"
jp_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/J_P_word.csv?raw=true"
nyt_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/NYT_word.csv?raw=true"

In [5]:
import re


def clean_text(text):
    # Normalize all types of single and double quotation marks to standard forms
    text = re.sub(r"[‘’`]", "'", text)  # Convert all single quote variations to '
    text = re.sub(r"[“”]", '"', text)  # Convert all double quote variations to "

    # remove any and all special characters since it will not be useful for our analysis
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    return text

In [6]:
import pandas as pd

# load the data
aj_df = pd.read_csv(aj_url)
bbc_df = pd.read_csv(bbc_url)
jp_df = pd.read_csv(jp_url)
nyt_df = pd.read_csv(nyt_url)

Here is the part you will need the model so I assume there is a variable called `model` which is the calssifier model you trained in the previous part.

In [11]:
def extract_all_sentences(df):
    # this will return a dict with key the id of the article "aj_1" for example
    # and a list of all the sentences in the article

    all_sentences = []

    for index, row in df.iterrows():
        text = row["document"]
        # TODO - ask gpt for a smarter sentence extratctor
        sentences = re.split(r"[.!?]", text)
        sentences = [sentence for sentence in sentences if sentence != ""]
        # clean the sentences
        sentences = [clean_text(sentence) for sentence in sentences]

        # for all sentence in sentences add to df
        for sentence in sentences:
            all_sentences.append({"id": row["id"], "document": sentence})

    return all_sentences

In [12]:
# for each of the dataframes, extract all the sentences

aj_sentences = extract_all_sentences(aj_df)
bbc_sentences = extract_all_sentences(bbc_df)
jp_sentences = extract_all_sentences(jp_df)
nyt_sentences = extract_all_sentences(nyt_df)

# the sentences are now extracted and cleaned
# in the following format {id: [sentence1, sentence2, ...]}

In [14]:
aj_df = pd.DataFrame(aj_sentences)
bbc_df = pd.DataFrame(bbc_sentences)
jp_df = pd.DataFrame(jp_sentences)
nyt_df = pd.DataFrame(nyt_sentences)

In [15]:
aj_df.head()

,id,document
0,aj_1,pope renews call for gaza ceasefire release o...
1,aj_1,pope francis has renewed calls for an immedia...
2,aj_2,biden is still the best us president israel co...
3,aj_2,united states president ronald reagans order ...
4,aj_3,israeli air strikes continue across gaza as tr...


In [17]:
df = pd.DataFrame(columns=["id", "document", "pro-israeli", "pro-palestinan", "neutral", "anti-isreali", "anti-palestinian", "majority_class"])

In [18]:
# append aj_df, ... into df (keeping the col)

df = pd.concat([df, aj_df], ignore_index=True)
df = pd.concat([df, bbc_df], ignore_index=True)
df = pd.concat([df, jp_df], ignore_index=True)
df = pd.concat([df, nyt_df], ignore_index=True)

df.head()

,id,document,pro-israeli,pro-palestinan,neutral,anti-isreali,anti-palestinian,majority_class
0,aj_1,pope renews call for gaza ceasefire release o...,NaN,NaN,NaN,NaN,NaN,NaN
1,aj_1,pope francis has renewed calls for an immedia...,NaN,NaN,NaN,NaN,NaN,NaN
2,aj_2,biden is still the best us president israel co...,NaN,NaN,NaN,NaN,NaN,NaN
3,aj_2,united states president ronald reagans order ...,NaN,NaN,NaN,NaN,NaN,NaN
4,aj_3,israeli air strikes continue across gaza as tr...,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.shape

(48877, 8)

In [ ]:
import torch
from transformers import DistilBertTokenizer

# Assuming the SentimentClassifier class is already defined as provided

def one_hot_to_class(one_hot_vector):
  map_class = {0: 'pro-israeli', 1: 'pro-palestinan', 2: 'neutral', 3: 'anti-isreali', 4: 'anti-palestinian'}

  # Find the index of the maximum value in the one-hot encoded vector
  class_index = torch.argmax(one_hot_vector).item()

  # Get the string representation of the class from the map_class dictionary
  class_string = map_class[class_index]

  return class_string


# Initialize the model and tokenizer
# model = SentimentClassifier()
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Function to classify a sentence and get probabilities
def classify_sentence_prob(sentence):
    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=512)

    # Get the model's output
    with torch.no_grad():
        logits = model(inputs['input_ids'], inputs['attention_mask'])

    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(logits, dim=1)

    # get the str of the calss based on the one-hot-encoded vector
    predicted_class = torch.argmax(probabilities, dim=1).item()

    # Create a one-hot encoded vector
    one_hot_vector = torch.zeros(probabilities.size(1))
    one_hot_vector[predicted_class] = 1

    return probabilities, one_hot_to_class(one_hot_vector)

In [ ]:
# now we wil use the model to classify each sentences for each article
# and we will classify the article based on the majority of the sentences

# TODO: load the model

# df is the new dataframe with id, class1, ..., calss5, majority_class
# the classi will contain the number of sentences in the article that belong to that class



for article_id, sentences in df.items():
    # classify the sentences
    # and add 1 to the class of the sentence
    pass

In [ ]:
# save df into a csv file for analysis

df.to_csv("classified_articles.csv", index=False)

In [ ]:
# analysis ...